In [ ]:
!pip install --upgrade pip setuptools
!pip install pydub
!pip install whisper
!pip install pytube
!pip install git+https://github.com/openai/whisper.git
!sudo apt update && sudo apt install ffmpeg
!pip install pyspellchecker
!pip install pyTelegramBotAPI

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3109, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2902, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 180, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 245, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 444, in run
    conflicts = self._determine_conflicts(to_install)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 

In [1]:
from pytube import YouTube
import requests
import io
import os
import json
from nltk.tokenize import sent_tokenize, word_tokenize
from spellchecker import SpellChecker
import re
import nltk
nltk.download('punkt')
from IPython.display import Audio
import telebot

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
def check_input(user_input):
    # Проверяем, является ли введенное значение ссылкой
    url_pattern = re.compile(
        r'^(http:\/\/www\.|https:\/\/www\.|http:\/\/|https:\/\/)?[a-z0-9]+([\-\.]{1}[a-z0-9]+)*\.[a-z]{2,5}(:[0-9]{1,5})?(\/.*)?$',
        re.IGNORECASE # Игнорирование регистра
    )

    # Проверяем, является ли введенное значение путем к аудиофайлу
    audio_extensions = ['.mp3', '.wav', '.flac', '.aac']  # список расширений аудиофайлов
    is_audio = any(user_input.lower().endswith(ext) for ext in audio_extensions)

    if re.match(url_pattern, user_input):
        print("Введенная информация является ссылкой.")
        audio_data, path_to_file = download_audio_from_link(user_input)
        text = audio_to_text(path_to_file)
        return text
    elif is_audio:
        print("Введенная информация - это путь к аудиофайлу.")
        text = audio_to_text(user_input)
        return text
    else:
        print("Неверный формат ввода. Пожалуйста, введите ссылку или путь к аудиофайлу.")

In [3]:
def download_audio_from_link(link):
    try:
        video = YouTube(link)

        if not video:
            print("Видео не найдено.")
            return None, None

        streams = video.streams.filter(only_audio=True).order_by('abr').desc()

        if streams:
            # Выбираем аудио-поток с наивысшим битрейтом
            highest_quality_audio = streams[0]
            print(f"Загружаем аудио максимального качества: {highest_quality_audio.title}")

            # Очищаем имя файла от недопустимых символов
            clean_title = re.sub(r'[\\/*?:"<>|]', '_', video.title)

            # Формируем путь к файлу для сохранения аудио
            file_path = f"/content/Untitled Folder/{clean_title}.mp3"

            # Проверяем наличие файла по указанному пути
            if os.path.exists(file_path):
                print("Файл уже существует. Выберите другое имя файла или место для сохранения.")
                return None, None

            # Загружаем аудио в файл
            highest_quality_audio.download(filename=file_path)

            # Считываем содержимое файла в переменную
            with open(file_path, 'rb') as file:
                audio_content = file.read()

            print("Аудио максимального качества успешно загружено в переменную!")
            return audio_content, file_path
        else:
            print("Аудио не найдено для данного видео.")
            return None, None
    except Exception as e:
        print(f"Произошла ошибка: {str(e)}")
        return None, None

# Пример использования
video_link = "https://www.youtube.com/watch?v=IdTCbSQIXD4"
audio_data, path_to_file = download_audio_from_link(video_link)

Загружаем аудио максимального качества: Я одинокий человек;)
Файл уже существует. Выберите другое имя файла или место для сохранения.


In [4]:
def audio_to_text(path_to_audio):
    text = !whisper '{path_to_audio}' --model base --language ru
    text = [i for i in text]
    text = ' '.join(text)
    return text

In [5]:
def text_preprocess(text):
  spell = SpellChecker(language='ru')
  corrected_text = spell.correction(text)

# Извлечение текста из таймкодов
  text_only = re.sub(r'\[\d{2}:\d{2}\.\d+\s*-->\s*\d{2}:\d{2}\.\d+\]\s*', '', corrected_text)

# Разделение на предложения
  sentences = sent_tokenize(text_only, language='russian')

# Токенизация и удаление пунктуации
  tokens = []
  for sentence in sentences:
      words = word_tokenize(sentence.lower(), language='russian')
      words = [word for word in words if word.isalnum()]  # Оставляем только буквенно-цифровые токены
      tokens.append(words)

  return tokens

In [6]:
def prompt_generation(text):

# Обновление поля "text" в сообщении в объекте prompt
  prompt = {
    "modelUri": "gpt://идентификатор папчки Яндекс Клауд/summarization",
    "completionOptions": {
      "stream": False,
      "temperature": 0.1,
      "maxTokens": "2000"
    },
    "messages": [
      {
            "role": "system",
            "text": "Ты должен сократить переданный тебе текст."
      },
      {
        "role": "user",
        "text": "Здесь будет обработанный текст."
      }
    ]
  }

  url = "https://llm.api.cloud.yandex.net/foundationModels/v1/completion"
  headers = {
      "Content-Type": "application/json",
      "Authorization": "Api-Key YandexGPT"
  }

  prompt["messages"][1]["text"] = text

  return prompt


In [7]:
def get_model_res(prompt, text):

  url = "https://llm.api.cloud.yandex.net/foundationModels/v1/completion"
  headers = {
      "Content-Type": "application/json",
      "Authorization": "Api-Key YandexGPT"
  }
  response = requests.post(url, headers=headers, json=prompt_generation(text))
  response_text = response.text
  response_dict = json.loads(response_text)

# Извлечение текста ответа модели
  model_response = response_dict["result"]["alternatives"][0]["message"]["text"]
  return model_response

In [17]:
def main(user_input):

  text = check_input(user_input)
  tokens = text_preprocess(text)
  prompt = prompt_generation(tokens)
  res = get_model_res(prompt, text)
  return res

In [18]:
TOKEN = 'Токен бота'
bot = telebot.TeleBot(TOKEN)

In [19]:
# Handle '/start'
@bot.message_handler(commands=['start'])
def start(message):
    bot.reply_to(message, """\n
Привет! Я PSVL v0.1, Моя задача показывать вам краткое текстовое содержание видео.\n
Вставьте ссылку на видео с YouTube\n
""")
@bot.message_handler(func=lambda message: True)

def take_input(message):
    user_input = message.text
    response = main(user_input)
    bot.reply_to(message, response)

In [20]:
bot.polling()

Введенная информация является ссылкой.
Загружаем аудио максимального качества: НА ЧТО СПОСОБЕН ИСКУССТВЕННЫЙ ИНТЕЛЛЕКТ ОТ OPENAI? @KosmoStory
Аудио максимального качества успешно загружено в переменную!
Введенная информация является ссылкой.
Загружаем аудио максимального качества: Не смейтесь с людей!
Файл уже существует. Выберите другое имя файла или место для сохранения.
Введенная информация является ссылкой.
Загружаем аудио максимального качества: Подробно про Apple Дневник
Аудио максимального качества успешно загружено в переменную!
